In [2]:
import scrapy
from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule
from urllib.parse import urlparse
from scrapy.utils.response import open_in_browser
from scrapy.crawler import CrawlerProcess
from urllib import parse
from os import path
from scrapy.http.response.html import HtmlResponse
import multiprocessing
from typing import List

In [4]:
class NewsSpider(CrawlSpider):

    name = 'crawler_pagina12'
    # solo descargar paginas desde estos dominios
    allowed_domains = ('www.pagina12.com.ar','pagina12.com.ar')
    
    rules = (
        # Ejemplo de regla de scrapping:
        # solo bajar las paginas cuya url incluye "/secciones", pero no aquellas cuya url include "/catamarca12" o "/dialogo".
        # Ud. tiene que modificar esta regla para bajar solo indice + noticias de pagina 12.
        # scrappy normaliza las urls para no descargarlas 2 veces la misma pagina con distinta url.
        Rule(LinkExtractor(allow=r'.+/secciones/.+',deny='.+(/catamarca12|/dialogo).+',
               deny_domains=['auth.pagina12.com.ar'], canonicalize=True,
               deny_extensions=['7z', '7zip', 'apk', 'bz2', 'cdr,' 'dmg', 'ico,' 'iso,' 'tar', 'tar.gz','pdf','docx', 'jpg', 'png', 'css', 'js']),
               callback='parse_response', follow=True),
     )
     
    # configuracion de scrappy, ver https://docs.scrapy.org/en/latest/topics/settings.html
    # la var de clase debe llamarse "custom settings"
    custom_settings = {
      # mentir el user agent
     'USER_AGENT': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
     'LOG_ENABLED': True,
     'LOG_LEVEL': 'INFO',
      # no descargar paginas mas alla de 1 link desde la pagina de origen
     'DEPTH_LIMIT': 2,
      # ignorar robots.txt (que feo eh)
     'ROBOTSTXT_OBEY': False,
      # esperar entre 0.5*DOWNLOAD_DELAY y 1.5*DOWNLOAD_DELAY segundo entre descargas
     'DOWNLOAD_DELAY': 1,
     'RANDOMIZE_DOWNLOAD_DELAY': True
    }

    def __init__(self, save_pages_in_dir='.', *args, **kwargs):
          super().__init__(*args, **kwargs)
          # guardar el directorio en donde vamos a descargar las paginas
          self.basedir = save_pages_in_dir
    
    def parse_response(self, response:HtmlResponse):
          """
          Este metodo es llamado por cada url que descarga Scrappy.
          response.url contiene la url de la pagina,
          response.body contiene los bytes del contenido de la pagina.
          """
          # el nombre de archivo es lo que esta luego de la ultima "/"
          html_filename = path.join(self.basedir,parse.quote(response.url[response.url.rfind("/")+1:]))
          if not html_filename.endswith(".html"):
              html_filename+=".html"
          print("URL:",response.url, "Pagina guardada en:", html_filename)
          # sabemos que pagina 12 usa encoding utf8
          with open(html_filename, "wt") as html_file:
              html_file.write(response.body.decode("utf-8"))

In [3]:
path.join(self.basedir,parse.quote(response.url[response.url.rfind("/")+1:]))

NameError: name 'self' is not defined

In [5]:
def start_crawler(page_number:int, save_pages_in_dir:str, start_urls:List[str]):
    crawler = CrawlerProcess()
    crawler.crawl(NewsSpider, save_pages_in_dir = save_pages_in_dir, start_urls = start_urls)
    crawler.start()

In [16]:
print(__name__)

__main__


In [7]:
import os

In [14]:
os.getcwd()

'/Users/achain/Documents/github/web-mining'

In [59]:
# -*- coding: utf-8 -*-
"""
Este script transforma un grupo de paginas html, agrupadas en directorios, a 1 directorio por categoria, en un dataset para entrenar.
Espera que haya 1 directorio por categoria dentro del directorio padre cuyo path esta en la variable DIR_BASE_CATEGORIAS. Usa el nombre del directorio como nombre de la categoria.
"""
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import re
import os
import joblib
from typing import Pattern, Optional, List, Tuple
#from tokenizers import tokenizador, tokenizador_con_stemming

#STOPWORDS_FILE = "stopwords_es.txt"
#STOPWORDS_FILE_SIN_ACENTOS = "stopwords_es_sin_acentos.txt"
# Este es el path COMPLETO del directorio que contiene a 1 subdirectorio por cada categoria; adentro de esos subdirs estan los html
DIR_BASE_CATEGORIAS = '/Users/achain/Documents/github/web-mining/pages/eco_content'

# este es el texto que tiene que aparecer en las notas, antes del texto de la nota
MARCADOR_COMIENZO_INTERESANTE="class=\"article-text\""
# este es el texto que tiene que aparecer en las notas, despues del texto de la nota
MARCADOR_FIN_INTERESANTE="div class=\"share"
extractor_de_parte_de_html_que_interesa = re.compile(re.escape(MARCADOR_COMIENZO_INTERESANTE) + "(.+)" + re.escape(MARCADOR_FIN_INTERESANTE))



In [34]:
def extraer_parte_que_interesa_de_html(regex:Pattern, texto:str) -> Optional[str]:
    """
    Usa una expresion regular con 1 grupo de captura para extraer una parte de un texto.
    :param regex:
    :param texto:
    :return: El texto extraido. Si no pudo extraer nada, retorna None.
    """
    #quitar todos los fines de linea para que la regex funcione sin importar como estaba dividido el html
    match = regex.search(texto.replace("\n",""))
    return match.group(1) if match is not None else None


def pasar_html_a_texto(html_doc:str) -> Optional[str]:
    """
    Recibe el HTML de una nota, corta una parte del html usando la regex 'extractor_de_parte_de_html_que_interesa', y y retorna el texto de esa parte, descartando todos los tags de html.
    :param html_doc:  HTML de una nota
    :return: El texto extraido de la nota, o None si no encontro texto para extraer.
    """
    html_que_interesa = extraer_parte_que_interesa_de_html(extractor_de_parte_de_html_que_interesa, html_doc)
    if html_que_interesa is not None:
        # beautifulsoup hace todo el trabajo de ignorar los tags de html y convertir las entidades (p.ej. &ntilde;) a letras (p.ej. ñ)
        extractor_html = BeautifulSoup(html_que_interesa, 'html.parser')
        texto = extractor_html.get_text(separator=" ", strip=True)
        return texto
    else:
        return "error"

In [28]:


def leer_archivo(path:str) -> str:
    return open(path,"rt").read()


In [24]:
for archivo_html in os.listdir(DIR_BASE_CATEGORIAS):
    path_completo_html = os.path.join(DIR_BASE_CATEGORIAS, archivo_html)

In [25]:
path_completo_html

'/Users/achain/Documents/github/web-mining/pages/eco_content/580242-fuerte-caida-del-dolar-blue'

In [65]:
htmls = []

for archivo_html in os.listdir(DIR_BASE_CATEGORIAS):
    path_completo_html = os.path.join(DIR_BASE_CATEGORIAS, archivo_html)
    if os.path.isfile(path_completo_html):
        try:
            with open(path_completo_html, "r", encoding="UTF-8") as file:
                texto = pasar_html_a_texto(file.read())
                if texto is not None:
                    htmls.append(texto)
                else:
                    print("CUIDADO! No fue posible extraer el texto de la nota del archivo {}".format(path_completo_html))
        except Exception as e:
            print("Error while processing {}: {}".format(path_completo_html, str(e)))



Error while processing /Users/achain/Documents/github/web-mining/pages/eco_content/.DS_Store: 'utf-8' codec can't decode byte 0xaf in position 1149: invalid start byte


In [60]:
pasar_html_a_texto(leer_archivo('/Users/achain/Documents/github/web-mining/pages/eco_content/580035-dolar-blue-hoy-dolar-hoy-a-cuanto-cotizan-el-viernes-18-de-a.html'))

'> La cotización del dólar blue alcanzó hoy los $710,00 y $720,00 para la compra y venta, respectivamente. En relación de la anterior jornada hábil, el blue continuó con tendencia a la baja, ya que los valores de ese día fueron: $750,00 para la compra y $760,00 para la venta. El valor del dólar oficial fue de $356,57 para la compra y $366,57 para la venta. Con estos números la variación situó la brecha entre la cotización del dólar oficial y el dólar blue a 99.12%. Cotización del dólar blue y el dólar oficial Actualizado: viernes 18 de agosto, 19:05 Compra venta Dólar Blue $710,00 $720,00 Dólar oficial $356,57 $366,57 Hay actualizaciones de la nota! 19:05 |\xa018/08/2023 |\xa03 horas atrás Actualización 18/08/2023 - 16:33 dólar blue cerró 99.12% arriba del dólar oficial: $720,00 para la compra y $710,00 para la venta. Dólar oficial $356,57 y $366,57. Respecto del anterior día hábil el blue registró una tendencia a la baja: donde cotizaba a $750,00 para la compra y $760,00 para la venta

In [39]:
html_que_interesa = extraer_parte_que_interesa_de_html(extractor_de_parte_de_html_que_interesa, leer_archivo('/Users/achain/Documents/github/web-mining/pages/eco_content/580035-dolar-blue-hoy-dolar-hoy-a-cuanto-cotizan-el-viernes-18-de-a.html'))

In [ ]:
from Opti 

In [41]:
print(leer_archivo('/Users/achain/Documents/github/web-mining/pages/eco_content/580035-dolar-blue-hoy-dolar-hoy-a-cuanto-cotizan-el-viernes-18-de-a.html'))

<!doctype html><html amp lang="es" i-amphtml-layout i-amphtml-no-boilerplate transformed="self;v=1"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width"><link rel="preconnect" href="https://fonts.gstatic.com" crossorigin><link rel="dns-prefetch" href="https://fonts.gstatic.com"><meta name="robots" content="max-image-preview:large"><meta name="cXenseParse:oct-channel" content="amp"><meta name="cXenseParse:url" content="https://www.pagina12.com.ar/580035-dolar-blue-hoy-dolar-hoy-a-cuanto-cotizan-el-viernes-18-de-a?ampOptimize=1"><meta property="article:opinion" content="false"><meta property="article:content_tier" content="free"><meta property="article:location" content="city:buenos aires"><meta name="amp-script-src" content="sha384-Mk6AXUUavrHTy4AJvbBr-OZb_VIRsdKTKk-MEFWcw2yO731_1Z1P_YIOgGnzUUFt sha384-l1FdVuJN3K6jD8EaXmJ69zZzv2iEkVXvirqFWqgFtY3Hy67wob0s32Rurrp0L_AT sha384-9wELknc8CBPik5j3srsKJu8YpM2A1oAjFQFYhHjZWdLO18jOGVpSTnMsGFMokLzy"><meta property="descrip

In [ ]:






def htmls_y_target(dir_de_1_categoria:str) -> Tuple[List[str],List[str]]:
    """
    Lee todos los archivos html en el directorio, y retorna un par([lista de html],[lista de categoria]).
    El nombre del directorio se usa como categoria.
    :param dir_de_1_categoria: Path completo del directorio de donde leer archivos html.
    :return: un par([lista de html],[lista de categorias]) donde a cada html le corresponde la misma categoria, asignada en base al nombre del directorio..
    """

